In [ ]:
# default_exp core

In [2]:
#hide
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [8]:
#hide
!pip install nbdev
import os
# define pathway to the weights
weights_filename = '21-2-25 1k-digits YOLOv5-weights.pt'
resource_folder = "/content/drive/MyDrive/Coding/Roboflow/try it out"
weights_path = os.path.join(resource_folder, weights_filename)

     |████████████████████████████████| 51kB 2.5MB/s 
     |████████████████████████████████| 61kB 3.7MB/s 
     |████████████████████████████████| 51kB 3.1MB/s 


# module name here

> API details.

In [9]:
#hide
from nbdev.showdoc import *

In [10]:
import pandas as pd

class ProcessedResult:
  def __init__(self, alias, filename = None, data_in_memory=None):
    if data_in_memory is None:
      with open(filename, "r") as infile:
        data = json.load(infile)
    if filename is None and data is None:
      raise
      
    # instantiate the predictions dataframe
    self.data = data
    self.processed = data["processed"]
    df = pd.DataFrame.from_dict(data["df"])
    
    self.alias = alias
    self.filename = filename

    print( len(self.data),len(self.processed))
    